Below is code with a link to a happy or sad dataset which contains 80 images, 40 happy and 40 sad. Create a convolutional neural network that trains to 100% accuracy on these images, which cancels training upon hitting training accuracy of >.999

Hint -- it will work best with 3 convolutional layers.

In [ ]:
import tensorflow as tf
from tensorflow import keras
import os
import zipfile
from os import path, getcwd, chdir

# DO NOT CHANGE THE LINE BELOW. If you are developing in a local
# environment, then grab happy-or-sad.zip from the Coursera Jupyter Notebook
# and place it inside a local folder and edit the path to that location
path = "/content/drive/MyDrive/study/tf-specialization/course1_intro_to_ai_ml_dl/week 4/tmp2/happy-or-sad.zip"

zip_ref = zipfile.ZipFile(path, 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

In [ ]:
class myCallback(keras.callbacks.Callback):
  def on_epoch_end(self, epochs, logs={}):
    if(logs.get("accuracy") >= 0.999):
      print("Reached 99% training accuracy so stopping")
      self.model.stop_training = True

In [ ]:
# data dir
train_dir = "/tmp/h-or-s"

In [ ]:
# get data and load it
from keras.preprocessing import image

train_datagen = image.ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
    train_dir, 
    target_size = (150,150),
    class_mode = "binary"
)



In [ ]:
# create model

model = keras.models.Sequential([
    keras.layers.Conv2D(64, (3,3), activation="relu", input_shape=(150,150,3)),
    keras.layers.MaxPooling2D(2,2),
    keras.layers.Conv2D(128, (3,3), activation="relu"),
    keras.layers.MaxPooling2D(2,2),
    keras.layers.Conv2D(256, (3,3), activation="relu"),
    keras.layers.MaxPooling2D(2,2),

    keras.layers.Flatten(),
    keras.layers.Dense(512, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])

model.summary()

In [ ]:
# compile

model.compile(loss="binary_crossentropy", optimizer=keras.optimizers.RMSprop(lr=0.001), metrics=["accuracy"])

In [ ]:
callbacks = myCallback()
model.fit_generator(
    train_generator,
    epochs = 20,
    callbacks=[callbacks]
)

In [ ]:
model.save("ex4.h5")